# Fibre preserving symmetries in the phase-plane
*Date*: 2022-06-07,<br>
*Written by:* Johannes Borgqvist.<br>
Now, we study a general time-invariant two component system of ODEs:
\begin{align}
    \dfrac{\mathrm{d}u}{\mathrm{d}t}&=\omega_1(u,v),\\
    \dfrac{\mathrm{d}v}{\mathrm{d}t}&=\omega_2(u,v).\\
\end{align}
and we are interested in this type of infinitesimal generator of the Lie group:
\begin{equation}
X = \eta_1(u,v)\partial_u+\eta_2(u,v)\partial_v.
  \label{eq:generator_phase_plane}
\end{equation}
It turns out that for this type of generators there is a single linearised symmetry condition that corresponds to solving the following PDE:
\begin{equation}
 \omega_1^2 \dfrac{\partial\eta_2}{\partial u}+\omega_1\omega_2\left(\dfrac{\partial\eta_2}{\partial v}-\dfrac{\partial\eta_1}{\partial u}\right)-\omega_2^2\dfrac{\partial\eta_1}{\partial v}=\left(\dfrac{\partial\omega_2}{\partial u}\omega_1-\omega_2\dfrac{\partial\omega_1}{\partial u}\right)\eta_1+\left(\dfrac{\partial\omega_2}{\partial v}\omega_1-\omega_2\dfrac{\partial\omega_1}{\partial v}\right)\eta_2.
\label{eq:lin_sym_fibre}
\end{equation}
Now, we would like to see if we can find any solutions for $\eta_1(u,v)$ and $\eta_2(u,v)$ using polynomial ans\"atze, and we will try to solve these equations using *SymPy*. <br>

In particular, we will look at the Lotka-Volterra (LV) model, the Belusov-Zhabotinskii (BZ) model and the Brusselator model. They all have polynomial reaction terms $\omega_1(u,v)$ and $\omega_2(u,v)$.

# Import libraries

In [1]:
# Import sympy
from sympy import *
# Translate a string to symbolic expression
from sympy.parsing.sympy_parser import parse_expr
# Finding monomials
from sympy.polys.monomials import itermonomials, monomial_count
# Ordering monomials 
from sympy.polys.orderings import monomial_key
# For printing to a file
import sys
# Import mathematical function
import math as m
# To extract all subsets of the coefficient vector
#from itertools import chain, combinations
import itertools
# Import numpy as well
import numpy as np

# Functions


In [18]:
#----------------------------------------------------------------------------------
#----------------------------------------------------------------------------------
# FUNCTION 1: "create_tangent_ansatze"
#----------------------------------------------------------------------------------
#----------------------------------------------------------------------------------
# The function takes two inputs:
#1. The states in the list states, 
#2. The degree of the polynomial ("degree_polynomial").
# It returns three outputs:
# 1.The unknown coefficients in the tangential ansatze,
# 2. A list of all tangents ("eta_list"),
# 3. The list of all monomials.
# The function uses various functionalities in sympy and the polynomials are generated using the built in functions in the polys.monomial part of the sympy library. To automate the generation of the sympy functions and symbol, the "exec" command is used at multiple places in the function.
def create_tangential_ansatze(states,degree_polynomial):
    #----------------------------------------------------------------------------------
    # INITIALISATION: ALLOCATE MEMORY FOR OUR OUTPUT
    #----------------------------------------------------------------------------------
    # Calculate the number of variables and number of states
    num_of_states = len(states)
    # Allocate our to lists which will be the output:
    eta_list = [] # The function with the tangents        
    c = [] # The list of the coefficients
    polynomial_list = [] # The function with the tangents        
    #----------------------------------------------------------------------------------
    # STEP 1: GENERATE POLYNOMIALS IN THE TEMPORARY VARIABLES
    #----------------------------------------------------------------------------------
    # Generate all monomials for our polynomials
    M = list(itermonomials(states, degree_polynomial))
    # Sort the list
    M = sorted(M, key=monomial_key('lex', states))
    # Calculate the number of terms in each of the polynomials
    num_of_monomials = monomial_count(num_of_states,degree_polynomial)
    #----------------------------------------------------------------------------------
    # STEP 2: DEFINE THE POLYNOMIAL ANSATZE
    #----------------------------------------------------------------------------------
    # Allocate our common polynomial
    P_0 = symbols('P_0', cls=Function)
    # Loop over the states and allocate our polynomial
    for state_index in range(2*num_of_states):
        # Initialise it to zero
        P_0 = 0    
        # Loop over the number of terms
        for monomial_index,monomial in enumerate(M):
            # Define the current index of our coefficient
            index = monomial_index + state_index*num_of_monomials
            # Allocate a coefficient
            exec("c_%d = symbols(\'c_%d\') "%(index,index)) 
            # Add this coefficient to the set of unknowns
            exec("c.append(c_%d)"%(index))
            # Add this coefficent to our polynomial at hand
            P_0+=c[-1]*M[monomial_index]
        polynomial_list.append(P_0)
    # When we are done we add P_0 to our list of tangential ansatze
    eta_list.append(polynomial_list[0]/polynomial_list[1])
    eta_list.append(polynomial_list[2]/polynomial_list[3])
    # Return the output    
    return c, eta_list, M, polynomial_list
#----------------------------------------------------------------------------------
#----------------------------------------------------------------------------------
# FUNCTION 2: "lin_sym_phase_plane"
#----------------------------------------------------------------------------------
#----------------------------------------------------------------------------------
# This function sets up the linearised symmetry condition for the phase plane. 
# It takes three inputs:
# 1. The states,
# 2. The reaction terms,
# 3. The tangents which are rational functions,
# 4. The polynomials making up the tangents.
# It returns three outputs:
# 1. The linearised symmetry condition in a variable called lin_sym,
# 2. The system of equation stemming from the monomials being linearly independent,
# 3. The monomials that each equation stems from.
def  lin_sym_phase_plane(states,omega,polynomial_list):
    # Allocate memory for our outputs
    lin_sym = [] # The linearised symmetry condition
    eq_sys = [] # The system of equations
    monomial_sys = [] # The monomials each equation stems from
    #----------------------------------------------------------------
    # STEP 1: Calculate the partial derivatives of the rational functions
    #----------------------------------------------------------------
    # Here, we only calculate the nominators of the of our lovely derivatives
    der_1 = Derivative(polynomial_list[0],states[0]).doit()*polynomial_list[1]-Derivative(polynomial_list[1],states[0]).doit()*polynomial_list[0]
    der_2 = Derivative(polynomial_list[0],states[1]).doit()*polynomial_list[1]-Derivative(polynomial_list[1],states[1]).doit()*polynomial_list[0]
    der_3 = Derivative(polynomial_list[2],states[0]).doit()*polynomial_list[3]-Derivative(polynomial_list[2],states[0]).doit()*polynomial_list[3]
    der_4 = Derivative(polynomial_list[2],states[1]).doit()*polynomial_list[3]-Derivative(polynomial_list[2],states[1]).doit()*polynomial_list[3]
    #----------------------------------------------------------------
    # STEP 2: SETUP THE LINEARISED SYMMETRY CONDITION
    #----------------------------------------------------------------
    # The linearised symmetry condition is hard coded
    temp_eq = 0
    # The LHS
    temp_eq += (omega[0]**2)*der_3*(polynomial_list[1]**2)
    temp_eq += (omega[0]*omega[1])*((der_4*(polynomial_list[1]**2))-(der_1*(polynomial_list[3]**2)))
    temp_eq += -(omega[1]**2)*der_2*(polynomial_list[3]**2)
    # The RHS
    temp_eq += -(Derivative(omega[1],states[0]).doit()*omega[0]- omega[1]*Derivative(omega[0],states[0]).doit())*polynomial_list[0]*polynomial_list[1]*(polynomial_list[3]**2)
    temp_eq += -(Derivative(omega[1],states[1]).doit()*omega[0]- omega[1]*Derivative(omega[0],states[1]).doit())*polynomial_list[2]*polynomial_list[3]*(polynomial_list[1]**2)
    # Simplify our linearised symmetry condition by multiplying with the product of the squares of the 
    # polynomials in the denominators of the rational functions in the tangential ansatze
    temp_eq = expand(temp_eq)
    # Append our lovely equation
    lin_sym.append(temp_eq)
    #----------------------------------------------------------------
    # STEP 3: EXTRACT THE EQUATIONS STEMMING FROM THE LIN SYM
    #----------------------------------------------------------------    
    # Generate all monomials of a high degree
    M = list(itermonomials(states, 20))
    # Sort the list
    M = sorted(M, key=monomial_key('lex', states))
    # Loop through the monomials, and extract all coefficients that are non-zero
    for m_index,monomial in enumerate(M):
        # Calculate the coefficient at hand
        temp_eq = lin_sym[0].coeff(monomial)
        # Set all higher order monomials to zero
        for state_index,state in enumerate(states):
            temp_eq = temp_eq.subs(state,0)
        # If we have a non-zero equation we append it to our list of equations
        if temp_eq != 0:
            eq_sys.append(temp_eq)
            monomial_sys.append(monomial)
    # Return the output 
    return lin_sym,eq_sys,monomial_sys
#----------------------------------------------------------------------------------
#----------------------------------------------------------------------------------
# FUNCTION 3: "extract_generators"
#----------------------------------------------------------------------------------
#----------------------------------------------------------------------------------
# This function sets up the linearised symmetry condition for the phase plane. 
# It takes four inputs:
# 1. c the coefficients in the initial ansatze,
# 2. LHS being the coefficients we have solved for,
# 3. RHS being the value of these coefficients in terms of other coefficients,
# 4. eta being the list of the initial tangents.
# It returns a single output being the list of all generators as a tuple of the two 
# infinitesimals namely (eta_1,eta_2).
def extract_generators(c,LHS,RHS,eta):
    # Allocate memory of our output
    generator_list = []    
    #----------------------------------------------------------------
    # STEP 1: FIND THE CONSTANTS THAT APPEAR IN THE FINAL SOLUTION
    #----------------------------------------------------------------
    # See which constants that exists among the constants
    constants_final = []
    # Loop over all algebraic expressions
    for RHS_index,RHS_temp in enumerate(RHS):
            # Loop over all candidate constants
            for c_index,c_temp in enumerate(c):
                # See if the constant exists in the final solution
                if RHS_temp.coeff(c_temp) != 0:
                    # Add the constant
                    constants_final.append(c_temp)
    # Only save the unique ones
    constants_final = list(set(constants_final))
    #----------------------------------------------------------------
    # STEP 2: SUBSTITUTE CONSTANTS INTO THE TANGENTS
    #----------------------------------------------------------------
    # Allocate memory for the final tangents
    eta_final = eta
    # Loop through the tangents and substitute the values for the coefficients
    for eta_index,eta_temp in enumerate(eta_final):
    # Loop through the coefficients as well
        for c_index,LHS_temp in enumerate(LHS):
            eta_final[eta_index] = eta_final[eta_index].subs(LHS_temp,RHS[c_index])
        # Simplify in the end to clean it up
        eta_final[eta_index] = expand(eta_final[eta_index])
    #----------------------------------------------------------------
    # STEP 3: EXTRACT GENERATORS
    #----------------------------------------------------------------    
    # ALlocate memory for the trivial generator
    trivial_generator = eta_final
    trivial_generator_returned = []
    # Loop over all our coefficients and save our generators
    for constant_index,constant in enumerate(constants_final):
        # Append the current generator
        generator_list.append((factor(eta_final[0].coeff(constant)),factor(eta_final[1].coeff(constant))))
        # Substract the current generator from the trivial generator
        trivial_generator[0] = expand(trivial_generator[0] - eta_final[0].coeff(constant)*constant)
        trivial_generator[1] = expand(trivial_generator[1] - eta_final[1].coeff(constant)*constant)
    # Lastly, append the trivial generator as well
    if simplify(trivial_generator[0]) != 0 and simplify(trivial_generator[1])!= 0:
        trivial_generator_returned.append((simplify(trivial_generator[0]),simplify(trivial_generator[1])))
    # Return our generators
    return generator_list, trivial_generator_returned
#----------------------------------------------------------------------------------
#----------------------------------------------------------------------------------
# FUNCTION 4: "findsubsets"
#----------------------------------------------------------------------------------
#----------------------------------------------------------------------------------
def findsubsets(s, n):
    return list(itertools.combinations(s, n))
#----------------------------------------------------------------------------------
#----------------------------------------------------------------------------------
# FUNCTION 5: "solve_algebraic_system"
#----------------------------------------------------------------------------------
#----------------------------------------------------------------------------------
# This function solves an individual algebraic system stemming from plugging in a 
# coefficient vector into the original massive system of equations. 
# It takes four inputs:
# 1. coeff_vec the coefficients we are plugging in,
# 2. c the original coefficients,
# 3. c_subset send in the correspond,
# 3. eta the tangents of the model in question,
# 4. eq_sys being the full system of equations that we want to reduce.
# It returns a single output which is a text string of the calculated solutions. 
def solve_algebraic_system(coeff_vec,c,c_subset,eta,eq_sys):
    # Allocate memory for our text string 
    generator_string = ""
    # We start by creating a temporary tangent
    eta_temp = eta  
    # Substitute our candidate coefficient vector into our tangential ansätze
    for coeff_index,coeff_temp in enumerate(coeff_vec):
        # Substitute all zeroes here...
        if coeff_temp == 0:
            eta_temp[0] = eta_temp[0].subs(c[coeff_index],coeff_temp)
            eta_temp[1] = eta_temp[1].subs(c[coeff_index],coeff_temp)    
    # Extract the reduced system of equations
    eq_sys_reduced = []
    for eq_temp in eq_sys:
        temp_eq = eq_temp
        for coeff_index,coeff_temp in enumerate(coeff_vec):
            temp_eq = temp_eq.subs(c[coeff_index],coeff_temp)
        if temp_eq != 0:
            eq_sys_reduced.append(temp_eq)    
    # Solve the system
    if len(eq_sys_reduced)!=0:        
            solutions = solve(eq_sys_reduced,c_subset,set=True)
    # Allocate memory for the generators
    generators = []
    for RHS_index,RHS in enumerate(solutions[1]):
        eta_sol = eta_temp
        for c_index,coeff_temp in enumerate(RHS):
            eta_sol[0] = eta_sol[0].subs(solutions[0][c_index],coeff_temp)
            eta_sol[1] = eta_sol[1].subs(solutions[0][c_index],coeff_temp)
            if eta_sol[0]!=0 or eta_sol[1]!=0:    
                generators.append(eta_sol)
        if len(generators)!=0:
            for generator in generators:
                if generator[0]!= 0 and generator[0]!=nan and generator[1] != 0 and generator[1] != nan:
                    generator_string += "\\begin{align*}\n"
                    generator_string += "X_{}&=" + latex(generator[0]) + "\\partial_{u}+" + latex(generator[1]) + "\\partial_{v},\\\\\n"
                    generator_string += "\\end{align*}"
            
    # Return our generator string
    return generator_string

# The LV-model

Now, we are considering the Lotka-Volterra model:

\begin{equation}
  \begin{split}
    \dfrac{\mathrm{d}u}{\mathrm{d}\tau}&=u(1-v),\\
    \dfrac{\mathrm{d}v}{\mathrm{d}\tau}&=\alpha v(u-1).\\    
    \end{split}
  \label{eq:LV}
\end{equation}

### Set up the reaction terms

In [3]:
# Dependent variables
u,v = symbols('u v')
# Define our parameter a in the LV model
a = symbols('a')
# Define the states
states_LV = [u, v]
# Define our reaction terms
omega_LV = [u*(1-v), a*v*(u-1)]

### Generate tangential ansatze

In [4]:
c_LV, eta_LV, M_LV, poly_LV = create_tangential_ansatze(states_LV,2)
print("The monomials:")
print(latex(Matrix(len(M_LV),1,M_LV),mode='equation').replace("\\begin{equation}","\\begin{equation}\n\\mathbf{M}=").replace("\\end{equation}",".\n\\end{equation}"))
print("The unknown coefficients:")
print(latex(Matrix(len(c_LV),1,c_LV),mode='equation').replace("\\begin{equation}","\\begin{equation}\n\\mathbf{c}=").replace("\\end{equation}",".\n\\end{equation}"))
print("The tangential ansatze:")
print("\\begin{align*}")
for state_index,state in enumerate(states_LV):
    if state_index == len(states_LV)-1:
        print("\\eta_%d(%s,%s)&=%s.\\\\"%(state_index+1,latex(states_LV[0]),latex(states_LV[1]),latex(eta_LV[state_index])))
    else:
        print("\\eta_%d(%s,%s)&=%s,\\\\"%(state_index+1,latex(states_LV[0]),latex(states_LV[1]),latex(eta_LV[state_index])))
print("\\end{align*}")

The monomials:
\begin{equation}
\mathbf{M}=\left[\begin{matrix}1\\v\\v^{2}\\u\\u v\\u^{2}\end{matrix}\right].
\end{equation}
The unknown coefficients:
\begin{equation}
\mathbf{c}=\left[\begin{matrix}c_{0}\\c_{1}\\c_{2}\\c_{3}\\c_{4}\\c_{5}\\c_{6}\\c_{7}\\c_{8}\\c_{9}\\c_{10}\\c_{11}\\c_{12}\\c_{13}\\c_{14}\\c_{15}\\c_{16}\\c_{17}\\c_{18}\\c_{19}\\c_{20}\\c_{21}\\c_{22}\\c_{23}\end{matrix}\right].
\end{equation}
The tangential ansatze:
\begin{align*}
\eta_1(u,v)&=\frac{c_{0} + c_{1} v + c_{2} v^{2} + c_{3} u + c_{4} u v + c_{5} u^{2}}{c_{10} u v + c_{11} u^{2} + c_{6} + c_{7} v + c_{8} v^{2} + c_{9} u},\\
\eta_2(u,v)&=\frac{c_{12} + c_{13} v + c_{14} v^{2} + c_{15} u + c_{16} u v + c_{17} u^{2}}{c_{18} + c_{19} v + c_{20} v^{2} + c_{21} u + c_{22} u v + c_{23} u^{2}}.\\
\end{align*}


The monomials:
\begin{equation}
\mathbf{M}=\left[\begin{matrix}1\\v\\v^{2}\\u\\u v\\u^{2}\end{matrix}\right].
\end{equation}
The unknown coefficients:
\begin{equation}
\mathbf{c}=\left[\begin{matrix}c_{0}\\c_{1}\\c_{2}\\c_{3}\\c_{4}\\c_{5}\\c_{6}\\c_{7}\\c_{8}\\c_{9}\\c_{10}\\c_{11}\\c_{12}\\c_{13}\\c_{14}\\c_{15}\\c_{16}\\c_{17}\\c_{18}\\c_{19}\\c_{20}\\c_{21}\\c_{22}\\c_{23}\end{matrix}\right].
\end{equation}
The tangential ansatze:
\begin{align*}
\eta_1(u,v)&=\frac{c_{0} + c_{1} v + c_{2} v^{2} + c_{3} u + c_{4} u v + c_{5} u^{2}}{c_{10} u v + c_{11} u^{2} + c_{6} + c_{7} v + c_{8} v^{2} + c_{9} u},\\
\eta_2(u,v)&=\frac{c_{12} + c_{13} v + c_{14} v^{2} + c_{15} u + c_{16} u v + c_{17} u^{2}}{c_{18} + c_{19} v + c_{20} v^{2} + c_{21} u + c_{22} u v + c_{23} u^{2}}.\\
\end{align*}



### Setup the linearised symmetry condition

In [5]:
# Plug in the tangential ansatze and calculate the resulting equations
lin_sym_LV,eq_sys_LV,monomials_LV = lin_sym_phase_plane(states_LV,omega_LV,poly_LV)
# Print these equations
print("The equations stemming from the linearised symmetry conditions:")
print("\\begin{align*}")
for eq_index,eq_temp in enumerate(eq_sys_LV):
    if eq_index == len(eq_sys_LV)-1:
        print("%s:&%s=0.\\\\"%(latex(monomials_LV[eq_index]),latex(eq_temp)))
    else:
        print("%s:&%s=0,\\\\"%(latex(monomials_LV[eq_index]),latex(eq_temp)))
print("\\end{align*}")

The equations stemming from the linearised symmetry conditions:
\begin{align*}
v:&- a c_{0} c_{18}^{2} c_{6}=0,\\
v^{2}:&a^{2} c_{0} c_{18}^{2} c_{7} - a^{2} c_{1} c_{18}^{2} c_{6} + a c_{0} c_{18}^{2} c_{6} - a c_{0} c_{18}^{2} c_{7} - 2 a c_{0} c_{18} c_{19} c_{6} - a c_{1} c_{18}^{2} c_{6}=0,\\
v^{3}:&2 a^{2} c_{0} c_{18}^{2} c_{8} + 2 a^{2} c_{0} c_{18} c_{19} c_{7} - 2 a^{2} c_{1} c_{18} c_{19} c_{6} - 2 a^{2} c_{18}^{2} c_{2} c_{6} + a c_{0} c_{18}^{2} c_{7} - a c_{0} c_{18}^{2} c_{8} + 2 a c_{0} c_{18} c_{19} c_{6} - 2 a c_{0} c_{18} c_{19} c_{7} - 2 a c_{0} c_{18} c_{20} c_{6} - a c_{0} c_{19}^{2} c_{6} + a c_{1} c_{18}^{2} c_{6} - a c_{1} c_{18}^{2} c_{7} - 2 a c_{1} c_{18} c_{19} c_{6} - a c_{18}^{2} c_{2} c_{6}=0,\\
v^{4}:&4 a^{2} c_{0} c_{18} c_{19} c_{8} + 2 a^{2} c_{0} c_{18} c_{20} c_{7} + a^{2} c_{0} c_{19}^{2} c_{7} + a^{2} c_{1} c_{18}^{2} c_{8} - 2 a^{2} c_{1} c_{18} c_{20} c_{6} - a^{2} c_{1} c_{19}^{2} c_{6} - a^{2} c_{18}^{2} c_{2} c_{7} - 4 a^{2} c_{18} c_{19} c_

u^{2} v^{3}:&- 4 a^{2} c_{0} c_{10} c_{18} c_{19} + 2 a^{2} c_{0} c_{10} c_{18} c_{22} + 2 a^{2} c_{0} c_{10} c_{19} c_{21} + 2 a^{2} c_{0} c_{18}^{2} c_{8} + 2 a^{2} c_{0} c_{18} c_{19} c_{7} - 8 a^{2} c_{0} c_{18} c_{21} c_{8} - 4 a^{2} c_{0} c_{18} c_{22} c_{7} + 4 a^{2} c_{0} c_{18} c_{23} c_{8} - 4 a^{2} c_{0} c_{19} c_{21} c_{7} + 2 a^{2} c_{0} c_{19} c_{23} c_{7} + 2 a^{2} c_{0} c_{21}^{2} c_{8} + 2 a^{2} c_{0} c_{21} c_{22} c_{7} - 2 a^{2} c_{1} c_{11} c_{18} c_{19} - 2 a^{2} c_{1} c_{18} c_{19} c_{6} + 4 a^{2} c_{1} c_{18} c_{19} c_{9} + 4 a^{2} c_{1} c_{18} c_{22} c_{6} - 2 a^{2} c_{1} c_{18} c_{22} c_{9} + 4 a^{2} c_{1} c_{19} c_{21} c_{6} - 2 a^{2} c_{1} c_{19} c_{21} c_{9} - 2 a^{2} c_{1} c_{19} c_{23} c_{6} - 2 a^{2} c_{1} c_{21} c_{22} c_{6} + 2 a^{2} c_{10} c_{18} c_{19} c_{3} - 2 a^{2} c_{11} c_{18}^{2} c_{2} - 2 a^{2} c_{18}^{2} c_{2} c_{6} + 4 a^{2} c_{18}^{2} c_{2} c_{9} - 4 a^{2} c_{18}^{2} c_{3} c_{8} + 2 a^{2} c_{18}^{2} c_{5} c_{8} - 4 a^{2} c_{18} c_{19} c_{3} 

u^{3} v^{2}:&a^{2} c_{0} c_{10} c_{18}^{2} - 4 a^{2} c_{0} c_{10} c_{18} c_{21} + 2 a^{2} c_{0} c_{10} c_{18} c_{23} + a^{2} c_{0} c_{10} c_{21}^{2} + 2 a^{2} c_{0} c_{18} c_{21} c_{7} - 4 a^{2} c_{0} c_{18} c_{23} c_{7} - 2 a^{2} c_{0} c_{21}^{2} c_{7} + 2 a^{2} c_{0} c_{21} c_{23} c_{7} + 2 a^{2} c_{1} c_{11} c_{18}^{2} - 2 a^{2} c_{1} c_{11} c_{18} c_{21} - a^{2} c_{1} c_{18}^{2} c_{9} - 2 a^{2} c_{1} c_{18} c_{21} c_{6} + 4 a^{2} c_{1} c_{18} c_{21} c_{9} + 4 a^{2} c_{1} c_{18} c_{23} c_{6} - 2 a^{2} c_{1} c_{18} c_{23} c_{9} + 2 a^{2} c_{1} c_{21}^{2} c_{6} - a^{2} c_{1} c_{21}^{2} c_{9} - 2 a^{2} c_{1} c_{21} c_{23} c_{6} - 2 a^{2} c_{10} c_{18}^{2} c_{3} + a^{2} c_{10} c_{18}^{2} c_{5} + 2 a^{2} c_{10} c_{18} c_{21} c_{3} - a^{2} c_{11} c_{18}^{2} c_{4} + a^{2} c_{18}^{2} c_{3} c_{7} - a^{2} c_{18}^{2} c_{4} c_{6} + 2 a^{2} c_{18}^{2} c_{4} c_{9} - 2 a^{2} c_{18}^{2} c_{5} c_{7} - 4 a^{2} c_{18} c_{21} c_{3} c_{7} + 4 a^{2} c_{18} c_{21} c_{4} c_{6} - 2 a^{2} c_{18} c_{21} c_{4}

u^{3} v^{5}:&2 a^{2} c_{0} c_{10} c_{19} c_{20} - 4 a^{2} c_{0} c_{10} c_{20} c_{22} + 4 a^{2} c_{0} c_{19} c_{22} c_{8} + 4 a^{2} c_{0} c_{20} c_{21} c_{8} + 2 a^{2} c_{0} c_{20} c_{22} c_{7} - 8 a^{2} c_{0} c_{20} c_{23} c_{8} - 4 a^{2} c_{0} c_{22}^{2} c_{8} + 4 a^{2} c_{1} c_{11} c_{19} c_{20} - 2 a^{2} c_{1} c_{11} c_{20} c_{22} + 2 a^{2} c_{1} c_{18} c_{22} c_{8} - 2 a^{2} c_{1} c_{19} c_{20} c_{9} + 2 a^{2} c_{1} c_{19} c_{21} c_{8} - 4 a^{2} c_{1} c_{19} c_{23} c_{8} - 2 a^{2} c_{1} c_{20} c_{22} c_{6} + 4 a^{2} c_{1} c_{20} c_{22} c_{9} - 4 a^{2} c_{1} c_{21} c_{22} c_{8} + 2 a^{2} c_{1} c_{22} c_{23} c_{8} - 2 a^{2} c_{10} c_{18} c_{19} c_{2} + 4 a^{2} c_{10} c_{18} c_{2} c_{22} + 4 a^{2} c_{10} c_{19} c_{2} c_{21} - 2 a^{2} c_{10} c_{19} c_{2} c_{23} - 4 a^{2} c_{10} c_{19} c_{20} c_{3} + 2 a^{2} c_{10} c_{19} c_{20} c_{5} - 2 a^{2} c_{10} c_{2} c_{21} c_{22} + 2 a^{2} c_{10} c_{20} c_{22} c_{3} + 8 a^{2} c_{11} c_{18} c_{2} c_{20} + 4 a^{2} c_{11} c_{19}^{2} c_{2} - 4 a^{2}

u^{4} v^{3}:&2 a^{2} c_{0} c_{10} c_{18} c_{22} + 2 a^{2} c_{0} c_{10} c_{19} c_{21} - 4 a^{2} c_{0} c_{10} c_{19} c_{23} - 4 a^{2} c_{0} c_{10} c_{21} c_{22} + 2 a^{2} c_{0} c_{10} c_{22} c_{23} + 4 a^{2} c_{0} c_{18} c_{23} c_{8} + 2 a^{2} c_{0} c_{19} c_{23} c_{7} + 2 a^{2} c_{0} c_{21}^{2} c_{8} + 2 a^{2} c_{0} c_{21} c_{22} c_{7} - 8 a^{2} c_{0} c_{21} c_{23} c_{8} - 4 a^{2} c_{0} c_{22} c_{23} c_{7} + 2 a^{2} c_{0} c_{23}^{2} c_{8} - 2 a^{2} c_{1} c_{11} c_{18} c_{19} + 4 a^{2} c_{1} c_{11} c_{18} c_{22} + 4 a^{2} c_{1} c_{11} c_{19} c_{21} - 2 a^{2} c_{1} c_{11} c_{19} c_{23} - 2 a^{2} c_{1} c_{11} c_{21} c_{22} - 2 a^{2} c_{1} c_{18} c_{22} c_{9} - 2 a^{2} c_{1} c_{19} c_{21} c_{9} - 2 a^{2} c_{1} c_{19} c_{23} c_{6} + 4 a^{2} c_{1} c_{19} c_{23} c_{9} - 2 a^{2} c_{1} c_{21} c_{22} c_{6} + 4 a^{2} c_{1} c_{21} c_{22} c_{9} + 4 a^{2} c_{1} c_{22} c_{23} c_{6} - 2 a^{2} c_{1} c_{22} c_{23} c_{9} + 2 a^{2} c_{10} c_{18} c_{19} c_{3} - 4 a^{2} c_{10} c_{18} c_{19} c_{5} - 4 a^{2} c

u^{5} v:&4 a c_{0} c_{11} c_{18} c_{23} + 2 a c_{0} c_{11} c_{21}^{2} - 6 a c_{0} c_{11} c_{21} c_{23} + 2 a c_{0} c_{21} c_{23} c_{9} - 2 a c_{0} c_{23}^{2} c_{9} - 2 a c_{10} c_{11} c_{12} c_{18} + 2 a c_{10} c_{11} c_{12} c_{21} + 2 a c_{10} c_{11} c_{15} c_{18} - 2 a c_{10} c_{12} c_{21} c_{9} - 2 a c_{10} c_{12} c_{23} c_{6} + 2 a c_{10} c_{12} c_{23} c_{9} - 2 a c_{10} c_{15} c_{18} c_{9} - 2 a c_{10} c_{15} c_{21} c_{6} + 2 a c_{10} c_{15} c_{21} c_{9} + 2 a c_{10} c_{15} c_{23} c_{6} - 2 a c_{10} c_{17} c_{18} c_{6} + 2 a c_{10} c_{17} c_{18} c_{9} + 2 a c_{10} c_{17} c_{21} c_{6} + a c_{11}^{2} c_{12} c_{19} + a c_{11}^{2} c_{13} c_{18} - 2 a c_{11} c_{12} c_{19} c_{9} - 2 a c_{11} c_{12} c_{21} c_{7} - 2 a c_{11} c_{12} c_{22} c_{6} + 2 a c_{11} c_{12} c_{22} c_{9} + 2 a c_{11} c_{12} c_{23} c_{7} - 2 a c_{11} c_{13} c_{18} c_{9} - 2 a c_{11} c_{13} c_{21} c_{6} + 2 a c_{11} c_{13} c_{21} c_{9} + 2 a c_{11} c_{13} c_{23} c_{6} - 2 a c_{11} c_{15} c_{18} c_{7} - 2 a c_{11} c_{

u^{6} v^{2}:&2 a^{2} c_{0} c_{10} c_{21} c_{23} - 2 a^{2} c_{0} c_{10} c_{23}^{2} + a^{2} c_{0} c_{23}^{2} c_{7} - 2 a^{2} c_{1} c_{11} c_{18} c_{23} - a^{2} c_{1} c_{11} c_{21}^{2} + 4 a^{2} c_{1} c_{11} c_{21} c_{23} - a^{2} c_{1} c_{11} c_{23}^{2} - 2 a^{2} c_{1} c_{21} c_{23} c_{9} - a^{2} c_{1} c_{23}^{2} c_{6} + 2 a^{2} c_{1} c_{23}^{2} c_{9} + 2 a^{2} c_{10} c_{18} c_{21} c_{5} + 2 a^{2} c_{10} c_{18} c_{23} c_{3} - 4 a^{2} c_{10} c_{18} c_{23} c_{5} + a^{2} c_{10} c_{21}^{2} c_{3} - 2 a^{2} c_{10} c_{21}^{2} c_{5} - 4 a^{2} c_{10} c_{21} c_{23} c_{3} + 2 a^{2} c_{10} c_{21} c_{23} c_{5} + a^{2} c_{10} c_{23}^{2} c_{3} - 2 a^{2} c_{11} c_{18} c_{21} c_{4} + 4 a^{2} c_{11} c_{18} c_{23} c_{4} + 2 a^{2} c_{11} c_{21}^{2} c_{4} - 2 a^{2} c_{11} c_{21} c_{23} c_{4} - 2 a^{2} c_{18} c_{23} c_{4} c_{9} + 2 a^{2} c_{18} c_{23} c_{5} c_{7} - a^{2} c_{21}^{2} c_{4} c_{9} + a^{2} c_{21}^{2} c_{5} c_{7} + 2 a^{2} c_{21} c_{23} c_{3} c_{7} - 2 a^{2} c_{21} c_{23} c_{4} c_{6} + 4 a^{2} c_{21

### Divde the algebraic system into numerous sub systems

In [6]:
# Find all tuples of subsets
c_subsets_LV = findsubsets(c_LV, 6)
# Convert all elements to lists
c_subsets_LV = [list(subset_as_tuple) for subset_as_tuple in c_subsets_LV]
# Some prompts to the user
print("The number of subsets of size 6:\t%d"%(len(c_subsets_LV)))
print("The first subset of size 6:%s"%(str(c_subsets_LV[0])))
# Find all corresponding coefficient vectors
coeff_vec_LV = []
# Set the remaining coefficients to zero if they are not in the subsets
for index,c_subset_LV in enumerate(c_subsets_LV):
    c_list = list(np.zeros((len(c_LV)), dtype=int))
    for const_index,const in enumerate(c_LV):
        for sub_index,c_temp in enumerate(c_subset_LV):
            if const == c_temp:
                c_list[const_index] = const
                break
    # Append our list
    coeff_vec_LV.append(c_list)
# Print the first coefficient vector
print("The first coefficient vector:")
print(latex(Matrix(len(coeff_vec_LV[0]),1,coeff_vec_LV[0]),mode='equation'))

The number of subsets of size 6:	134596
The first subset of size 6:[c_0, c_1, c_2, c_3, c_4, c_5]
The first coefficient vector:
\begin{equation}\left[\begin{matrix}c_{0}\\c_{1}\\c_{2}\\c_{3}\\c_{4}\\c_{5}\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\end{matrix}\right]\end{equation}


The number of subsets of size 6:	134596
The first subset of size 6:(c_0, c_1, c_2, c_3, c_4, c_5)
The first coefficient vector:
\begin{equation}\left[\begin{matrix}c_{0}\\c_{1}\\c_{2}\\c_{3}\\c_{4}\\c_{5}\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\end{matrix}\right]\end{equation}

Next, we want to add a filtration step where we remove all unneccessary ansätze resulting in zero of infinite tangents. 
# Filtration step


In [7]:
# Allocate memory for the indices we want to remove
remove_indices = []
print(len(coeff_vec_LV))
print(len(c_subsets_LV))
print(len(remove_indices))

# Loop through our coefficient candidates and remove the ones resulting in unreasonable ansatze
for coeff_index, coeff_vec in enumerate(coeff_vec_LV):
    # Extract our four polynomials
    P1 = poly_LV[0]
    P2 = poly_LV[1]
    P3 = poly_LV[2]
    P4 = poly_LV[3]
    # Loop through the coefficients in c and substitute our candidate values
    for c_index, coeff_temp in enumerate(c_LV):
        P1 = P1.subs(coeff_temp,coeff_vec[c_index])
        P2 = P2.subs(coeff_temp,coeff_vec[c_index])
        P3 = P3.subs(coeff_temp,coeff_vec[c_index])
        P4 = P4.subs(coeff_temp,coeff_vec[c_index])
    # Remove indices in the following cases
    if P1 == 0 and P3 == 0:
        remove_indices.append(coeff_index)
    elif P2 == 0:
        remove_indices.append(coeff_index)
    elif P4 == 0:
        remove_indices.append(coeff_index)
# Sort our remove indices
remove_indices.sort(reverse=True)
print("Number of indices to remove:\t%d"%(len(remove_indices)))
print("Total number of coefficients:\t%d"%(len(c_LV)))
print("Coefficients after removal:\t%d"%(len(c_LV)-len(remove_indices)))
# Lastly, remove the unneccessary coefficients
for remove_index in remove_indices:
    del coeff_vec_LV[remove_index]
    del c_subsets_LV[remove_index]

134596
134596
0
Number of indices to remove:	37126
Total number of coefficients:	24
Coefficients after removal:	-37102


# Solving the algebraic equations
Ok, so next we will try to solve one algebraic equation and see what comes of it. 

In [20]:
#print("Generating and solving all subsystems in our polynomial ansatze!")
# Solution string
solution_string = solve_algebraic_system(coeff_vec_LV[0],c_LV,c_subsets_LV[0],eta_LV,eq_sys_LV)
print(solution_string)
print(len(solution_string))
print("\nCALCULATIONS ARE FINISHED!")
# Now we are finished! 
#solutions_text_string += "\\end{align*}\n"
#print("The solutions were:")
#print(solutions_text_string)
#open text file
#text_file = open("./generators_LV.tex", "w")
#write string to file
#text_file.write(solutions_text_string)
#close file
#text_file.close()


0

CALCULATIONS ARE FINISHED!


In [ ]:
#print("Now, we will plug in all our generators into the linearised symmetry conditions. What we want to find is that it only spits out the value 0.")
#for index in range(len(generators_LV)):
    #lin_sym_control,eq_sys_temp,monomials_temp = lin_sym_phase_plane(states_LV,omega_LV,list(generators_LV[index]))
    #print("\t\tGenerator %d, value\t=\t%s"%(index+1,str(expand(lin_sym_control[0]))))

Ok, so now we have printed all these steps 

# The BZ-model
Now we are considering the BZ-model:
\begin{equation}
  \begin{split}
    \dfrac{\mathrm{d}u}{\mathrm{d}\tau}&=\dfrac{1}{\varepsilon}v-\dfrac{1}{\varepsilon}\left(\dfrac{1}{3}u^3-u\right),\\
    \dfrac{\mathrm{d}v}{\mathrm{d}\tau}&=-u.\\    
    \end{split}
  \label{eq:BZ}
\end{equation}




In [ ]:
# Dependent variables
#u,v = symbols('u v')
# Define our parameter a in the LV model
#e = symbols('e')
# Define the states
#states_BZ = [u, v]
# Define our reaction terms
#omega_BZ = [(1/e)*v-(1/e)*(((1/3)*u**3)-u),-u]
#omega_BZ = [(1/e)*v-(1/e)*((Rational(1,3)*u**3)-u),-u]
# Create the tangential ansatze
#c_BZ, eta_BZ, M_BZ = create_tangential_ansatze(states_BZ,5)
# Plug in the tangential ansatze and calculate the resulting equations
#lin_sym_BZ,eq_sys_BZ,monomials_BZ = lin_sym_phase_plane(states_BZ,omega_BZ,eta_BZ)
# Solve the system of equations for the unknown coefficients
#solutions_BZ = solve(eq_sys_BZ,c_BZ,set=True)
# Extract the LHS and RHS 
#LHS_BZ = solutions_BZ[0]
#RHS_BZ = list(list(solutions_BZ[1])[0])
# Calculate and extract all generators
#generators_BZ,trivial_generator = extract_generators(c_BZ,LHS_BZ,RHS_BZ,eta_BZ)
#print("The calculated infinitesimal generators of the Lie group:")
#print("\\begin{align*}")
#for gen_index in range(len(generators_BZ)):
    #if gen_index <len(generators_BZ)-1:
        #print("X_{%d}&=%s\\partial_u+%s\\partial_v,\\\\"%(gen_index+1,latex(generators_BZ[gen_index][0]),latex(generators_BZ[gen_index][1])))
    #else:
        #print("X_{%d}&=%s\\partial_u+%s\\partial_v.\\\\"%(gen_index+1,latex(generators_BZ[gen_index][0]),latex(generators_BZ[gen_index][1])))
#print("\\end{align*}")
#if len(trivial_generator)>0:
    #print("The trivial generator:")
    #trivial_str = "\\begin{equation}\nX_{0}="+latex(trivial_generator[0][0])+"\\partial_u+"+latex(trivial_generator[0][1])+"\\partial_v.\n\\end{equation}"
    #print("%s"%(trivial_str))
#print("The reaction terms:")
#print("\\begin{align*}\n\omega_1(u,v)&=%s,\\\\\n\omega_2(u,v)&=%s.\\\\\n\\end{align*}"%(latex(omega_BZ[0]),latex(omega_BZ[1])))

The calculated infinitesimal generators of the Lie group:
\begin{align*}
X_{1}&=\frac{u \left(u^{3} - 3 u - 3 v\right)}{3 e}\partial_u+u^{2}\partial_v,\\
X_{2}&=\frac{v^{2} \left(u^{3} - 3 u - 3 v\right)}{3 e}\partial_u+u v^{2}\partial_v,\\
X_{3}&=\frac{u v \left(u^{3} - 3 u - 3 v\right)}{3 e}\partial_u+u^{2} v\partial_v,\\
X_{4}&=\frac{u^{3} - 3 u - 3 v}{3 e}\partial_u+u\partial_v,\\
X_{5}&=\frac{v \left(u^{3} - 3 u - 3 v\right)}{3 e}\partial_u+u v\partial_v,\\
X_{6}&=\frac{u^{2} \left(u^{3} - 3 u - 3 v\right)}{3 e}\partial_u+u^{3}\partial_v.\\
\end{align*}
The reaction terms:
\begin{align*}
\omega_1(u,v)&=\frac{v}{e} - \frac{\frac{u^{3}}{3} - u}{e},\\
\omega_2(u,v)&=- u.\\
\end{align*}


# The Brusselator
Now we are considering the Brusselator model:
\begin{equation}
  \begin{split}
    \dfrac{\mathrm{d}u}{\mathrm{d}\tau}&=1-(b-1)u+au^2 v,\\
    \dfrac{\mathrm{d}v}{\mathrm{d}\tau}&=bu-au^2 v.\\
    \end{split}
  \label{eq:Brusselator}
\end{equation}




In [ ]:
# Dependent variables
#u,v = symbols('u v')
# Define our parameter a in the LV model
#a,b = symbols('a b')
# Define the states
#states_Brusselator = [u, v]
# Define our reaction terms
#omega_Brusselator = [1-((b-1)*u)+(a*(u**2)*v),b*u-(a*(u**2)*v)]
# Create the tangential ansatze
#c_Brusselator, eta_Brusselator, M_Brusselator = create_tangential_ansatze(states_Brusselator,5)
# Plug in the tangential ansatze and calculate the resulting equations
#lin_sym_Brusselator,eq_sys_Brusselator,monomials_Brusselator = lin_sym_phase_plane(states_Brusselator,omega_Brusselator,eta_Brusselator)
# Solve the system of equations for the unknown coefficients
#solutions_Brusselator = solve(eq_sys_Brusselator,c_Brusselator,set=True)
# Extract the LHS and RHS 
#LHS_Brusselator = solutions_Brusselator[0]
#RHS_Brusselator = list(list(solutions_Brusselator[1])[0])
# Calculate and extract all generators
#generators_Brusselator,trivial_generator = extract_generators(c_Brusselator,LHS_Brusselator,RHS_Brusselator,eta_Brusselator)
#print("The calculated infinitesimal generators of the Lie group:")
#print("\\begin{align*}")
#for gen_index in range(len(generators_Brusselator)):
    #if gen_index <len(generators_Brusselator)-1:
        #print("X_{%d}&=%s\\partial_u+%s\\partial_v,\\\\"%(gen_index+1,latex(generators_Brusselator[gen_index][0]),latex(generators_Brusselator[gen_index][1])))
    #else:
        #print("X_{%d}&=%s\\partial_u+%s\\partial_v.\\\\"%(gen_index+1,latex(generators_Brusselator[gen_index][0]),latex(generators_Brusselator[gen_index][1])))
#print("\\end{align*}")
#if len(trivial_generator)>0:
    #print("The trivial generator:")
    #trivial_str = "\\begin{equation}\nX_{0}="+latex(trivial_generator[0][0])+"\\partial_u+"+latex(trivial_generator[0][1])+"\\partial_v.\n\\end{equation}"
    #print("%s"%(trivial_str))
#print("The reaction terms:")
#print("\\begin{align*}\n\omega_1(u,v)&=%s,\\\\\n\omega_2(u,v)&=%s.\\\\\n\\end{align*}"%(latex(omega_Brusselator[0]),latex(omega_Brusselator[1])))

The calculated infinitesimal generators of the Lie group:
\begin{align*}
X_{1}&=- \frac{u \left(a u^{2} v - b u + u + 1\right)}{a}\partial_u+\frac{u^{2} \left(a u v - b\right)}{a}\partial_v,\\
X_{2}&=- \frac{v \left(a u^{2} v - b u + u + 1\right)}{a}\partial_u+\frac{u v \left(a u v - b\right)}{a}\partial_v,\\
X_{3}&=- \frac{u^{2} \left(a u^{2} v - b u + u + 1\right)}{a}\partial_u+\frac{u^{3} \left(a u v - b\right)}{a}\partial_v,\\
X_{4}&=- \frac{v^{2} \left(a u^{2} v - b u + u + 1\right)}{a}\partial_u+\frac{u v^{2} \left(a u v - b\right)}{a}\partial_v,\\
X_{5}&=- \frac{u v \left(a u^{2} v - b u + u + 1\right)}{a}\partial_u+\frac{u^{2} v \left(a u v - b\right)}{a}\partial_v.\\
\end{align*}
The reaction terms:
\begin{align*}
\omega_1(u,v)&=a u^{2} v - u \left(b - 1\right) + 1,\\
\omega_2(u,v)&=- a u^{2} v + b u.\\
\end{align*}


# Conclusion
I am not sure if all these generators are trivial, but let's present them and see what we think about it.